In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import scipy.stats

plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
df_sample = pd.read_csv('sample_site.csv')
df_scPDSI = pd.read_csv('annual_scPDSI.csv')
df_pre = pd.read_csv('annual_pre.csv')
df_temp = pd.read_csv('annual_temp.csv')
df_tmax = pd.read_csv('annual_Tmax.csv')
df_tmin = pd.read_csv('annual_Tmin.csv')
df_dtr = pd.read_csv('annual_dtr.csv')
df_pet = pd.read_csv('annual_pet.csv')

In [ ]:
def get_dry_wet_years(df):
    dry_years = {}
    wet_years = {}

    for site in df.index:
        dry_years[site] = []
        wet_years[site] = []

        for year in df.columns:
            value = df.loc[site, year]

            if -4 < value < -0.5:
                dry_years[site].append(year)
            elif 0.5 < value < 4:
                wet_years[site].append(year)

    return dry_years, wet_years

dry_years, wet_years = get_dry_wet_years(df_scPDSI.dropna().set_index('site'))
def get_dry_wet_years(df):
    dry_years = {}
    wet_years = {}

    for site in df.index:
        dry_years[site] = []
        wet_years[site] = []

        for year in df.columns:
            value = df.loc[site, year]

            if -4 < value < -0.5:
                dry_years[site].append(year)
            elif 0.5 < value < 4:
                wet_years[site].append(year)

    return dry_years, wet_years
def calculate_row_means(df1,df2):
    df_concat = pd.merge(df1,df2, on = 'site')
    df_concat.loc[:,'1901':].mean(axis= 1)
    return df_concat.loc[:,'1901':].mean(axis= 1).to_list()
dry_years, wet_years = get_dry_wet_years(df_scPDSI.dropna().set_index('site'))
list_Growth_loss = []
from tqdm import tqdm
site_list =pd.merge(df_sample,df_scPDSI.dropna().reset_index(),on = 'site')['site'].to_list()
for i in tqdm(range(len(site_list)), desc="Calculating Growth_loss"):
    site_i = site_list[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i =pd.read_csv(std_path)
    #Growth-no-drought
    wet_years_i  = [int(year) for year in wet_years[site_i]]
    wet_years_i_intersection = intersection_list_ordered(wet_years_i,df_rwl_i.index.to_list())
    Growth_no_dry = df_rwl_i.loc[wet_years_i_intersection,'std'].mean()
    #Growth-drought
    dry_years_i  = [int(year) for year in dry_years[site_i]]
    dry_years_i_intersection = intersection_list_ordered(dry_years_i,df_rwl_i.index.to_list())
    Growth_dry = df_rwl_i.loc[dry_years_i_intersection,'std'].mean()
    Growth_loss = (Growth_dry-Growth_no_dry)/Growth_no_dry
    list_Growth_loss.append(Growth_loss)
cs = pd.merge(df_sample,df_scPDSI.dropna().reset_index()['site'],on = 'site')
scpDSI_mean = calculate_row_means(cs,df_scPDSI)
pre_mean = calculate_row_means(cs,df_pre)
temp_mean = calculate_row_means(cs,df_temp)
tmax_mean = calculate_row_means(cs,df_tmax)
tmin_mean = calculate_row_means(cs,df_tmin)
dtr_mean = calculate_row_means(cs,df_dtr)
pet_mean = calculate_row_means(cs,df_pet)
mean_Groth_loss = {'site':site_list,'Growth_loss':list_Growth_loss,'pre_mean':pre_mean,
           'scpDSI_mean':scpDSI_mean,'temp_mean':temp_mean,'tmax_mean':tmax_mean,
                  'tmin_mean':tmin_mean,'dtr_mean':dtr_mean,'pet_mean':pet_mean
                  }
def calculate_confidence_interval(df, column):
    lower_bound = np.percentile(df[column], 2.5)
    upper_bound = np.percentile(df[column], 97.5)
    df_95 = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    mean_value = np.mean(df_95[column])
    print("95% Confidence Interval ({}): [{:.4f}, {:.4f}]".format(column, lower_bound, upper_bound))
    print("Mean ({}): {:.4f}".format(column, mean_value))
    return df_95
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
grouped = df_merge.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'Growth_loss')
group_arid95 = calculate_confidence_interval(group_arid, 'Growth_loss')
df_merge95 = pd.concat([group_humid95,group_arid95],axis=0)
f_statistic_dr, p_value_dr = f_oneway(group_humid95['Growth_loss'], group_arid95['Growth_loss'])
anova_results_dr['Growth_loss'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
colors = [(255/255, 0/255, 0/255),(0/255, 92/255, 230/255)]
plt.figure(figsize=(5,4.5),dpi=100)
#palette=['#8DCDD5','#E6846D']
alpha = 0.5 
sns.boxplot(x="AI_Class", y="Growth_loss", data=df_merge95
            ,palette=colors,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.ylim(-0.6,0.3)
plt.yticks([-0.6,-0.3,0,0.3]) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('Growth loss',fontsize=18)
plt.text(0.5, -0.5, '***', ha='center', va='center')
plt.text('Humid', -0.5,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', -0.5,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('Fig3b.jpg',dpi=300, bbox_inches='tight')
plt.show()
df_merge_95['log_ai_v3']= np.log10(df_merge_95['ai_v3'])
df_merge_95['log_ai_v3'].replace(-np.inf, np.nan, inplace=True)
df_merge_95.dropna(subset=['log_ai_v3'], inplace=True)
from scipy.stats import pearsonr
import statsmodels.api as sm
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(5,4.5),dpi=100)
species_groups = df_merge_95['Species_group'].unique()


color_map = {
    species_groups[0]: (0/255, 92/255, 230/255),
    species_groups[1]: (255/255, 0/255, 0/255)
}
for species in species_groups:
    df_group = df_merge_95[df_merge_95['Species_group'] == species]
    r, p = pearsonr(df_group['log_ai_v3'], df_group['Growth_loss'])
    X = df_group[['log_ai_v3']]
    y = df_group['Growth_loss']

    X_with_const = sm.add_constant(X) 
    model = sm.OLS(y, X_with_const).fit()
    intercept, slope = model.params
    predictions = model.get_prediction(X_with_const)
    predicted_mean = predictions.predicted_mean
    conf_int = predictions.conf_int()
    plt.scatter(df_group['log_ai_v3'], df_group['Growth_loss'], 
                alpha=0.06, label=f'{species} (r={r:.2f}, p={p:.2e})',
                color=color_map[species])
    plt.plot(df_group['log_ai_v3'], predicted_mean, color=color_map[species], linestyle='-')
    plt.fill_between(df_group['log_ai_v3'], conf_int[:, 0], conf_int[:, 1], color=color_map[species], alpha=0.2)
plt.yticks([-0.5, 0, 0.5])
plt.xlim(-1,1)
plt.xlabel('Log10(Aridity index)')
plt.ylabel('Growth loss',fontsize=18)
plt.legend(fontsize=10,loc=1)
plt.grid(True)
plt.savefig('FigS3c.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib
def calculate_vpd(temperature, actual_vap):
    s_vap = 6.108 * math.exp((17.27 * temperature) / (237.3 + temperature))
    vpd = s_vap - actual_vap
    return vpd
temperature = 21.350001
actual_vap = 13.516667  
vpd = calculate_vpd(temperature, actual_vap)
print(f"VPD: {vpd} kPa")
df_Temp = pd.read_csv('annual_temp.csv').set_index('site')
df_svp = pd.read_csv('annual_vap.csv').set_index('site')
def calculate_vpd1(df_Temp, df_svp):
    def saturation_vapor_pressure(temp):
        return 6.108 * math.exp((17.27 * temp) / (237.3 + temp))
    df_VPD = pd.DataFrame(index=df_Temp.index, columns=df_Temp.columns)
    for site in df_Temp.index:
        for year in df_Temp.columns:
            temp = df_Temp.loc[site, year]
            actual_vap = df_svp.loc[site, year]
            s_vap = saturation_vapor_pressure(temp)
            vpd = s_vap - actual_vap
            df_VPD.loc[site, year] = vpd
    
    return df_VPD
df_VPD = calculate_vpd1(df_Temp, df_svp)
df_VPD.to_csv('annual_VPD.csv')

In [ ]:
df_NPP = pd.read_csv('npp_values.csv')
df_NPP_Npp = df_NPP.filter(regex='_01_01_Npp$')
rename_dict = {col: col.split('_')[0] for col in df_NPP_Npp.columns}
df_NPP_Npp = df_NPP_Npp.rename(columns=rename_dict)
df_selected = pd.concat([df_NPP_Npp, df_NPP['site']], axis=1)
df_sample_NPP = pd.merge(df_sample,df_selected,on='site')
list_Growth_loss = []
list_siteNPP = []
from tqdm import tqdm
site_list =pd.merge(df_sample_NPP,df_scPDSI.dropna().reset_index(),on = 'site')['site'].to_list()
year2020 = list(range(2001, 2016))
for i in tqdm(range(len(site_list)), desc="Calculating Growth_loss"):
    site_i = site_list[i]
    wet_years_i  = [int(year) for year in wet_years[site_i]]
    wet_years_i_intersection = intersection_list_ordered(wet_years_i,year2020)
    wet_years_i_intersection_NPP= [str(year) for year in wet_years_i_intersection]
    Growth_no_dry = df_sample_NPP.set_index('site').loc[site_i,wet_years_i_intersection_NPP].mean()
    #Growth-drought
    dry_years_i  = [int(year) for year in dry_years[site_i]]
    dry_years_i_intersection = intersection_list_ordered(dry_years_i,year2020)
    dry_years_i_intersection_NPP = [str(year) for year in dry_years_i_intersection]
    Growth_dry = df_sample_NPP.set_index('site').loc[site_i,dry_years_i_intersection_NPP].mean()
    Growth_loss = (Growth_no_dry-Growth_dry)/Growth_no_dry
    list_Growth_loss.append(Growth_loss)
    list_siteNPP.append(site_i)
df_npp_GL = pd.DataFrame({'Growth_loss':list_Growth_loss,'site':list_siteNPP})
df_npp_GL1 = pd.merge(df_npp_GL,df_sample,on='site').dropna()
def calculate_confidence_interval(df, column):
    lower_bound = np.percentile(df[column], 2.5)
    upper_bound = np.percentile(df[column], 97.5)
    df_95 = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    mean_value = np.mean(df_95[column])
    print("95% Confidence Interval ({}): [{:.4f}, {:.4f}]".format(column, lower_bound, upper_bound))
    print("Mean ({}): {:.4f}".format(column, mean_value))
    return df_95
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
grouped = df_npp_GL1.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'Growth_loss')
group_arid95 = calculate_confidence_interval(group_arid, 'Growth_loss')
df_merge95 = pd.concat([group_humid95,group_arid95],axis=0)

f_statistic_dr, p_value_dr = f_oneway(group_humid95['Growth_loss'], group_arid95['Growth_loss'])
anova_results_dr['Growth_loss'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
df_species = pd.read_excel('40tree.xlsx')
species_dict = dict(zip(df_species['Tree_Speci'], df_species['Clade']))
df_merge95['Species_group'] = df_merge95['Tree_Speci'].map(species_dict)
df_merge95['Species_group'] = df_merge95['Species_group'].fillna('Unknown')
df_merge95['log_ai_v3']= np.log10(df_merge95['ai_v3'])
df_merge95['log_ai_v3'].replace(-np.inf, np.nan, inplace=True)
df_merge95.dropna(subset=['log_ai_v3'], inplace=True)
from scipy.stats import pearsonr
import statsmodels.api as sm
##### 设置图形大小
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(5,4.5),dpi=100)

# 获取唯一的物种组
species_groups = df_merge95['Species_group'].unique()

# 定义颜色蓝色'#8DCDD5','#E6846D'(255/255, 127/255, 127/255),(190/255, 210/255, 255/255)
color_map = {
    species_groups[0]: (0/255, 92/255, 230/255),
    species_groups[1]: (255/255, 0/255, 0/255)
}

for species in species_groups:
    df_group = df_merge95[df_merge95['Species_group'] == species]

    r, p = pearsonr(df_group['log_ai_v3'], df_group['Growth_loss'])

    X = df_group[['log_ai_v3']]
    y = df_group['Growth_loss']

    X_with_const = sm.add_constant(X)  
    model = sm.OLS(y, X_with_const).fit()

    intercept, slope = model.params

    predictions = model.get_prediction(X_with_const)
    predicted_mean = predictions.predicted_mean
    conf_int = predictions.conf_int()

    plt.scatter(df_group['log_ai_v3'], df_group['Growth_loss'], 
                alpha=0.06, label=f'{species} (r={r:.2f}, p={p:.2e})',
                color=color_map[species])

    plt.plot(df_group['log_ai_v3'], predicted_mean, color=color_map[species], linestyle='-')

    plt.fill_between(df_group['log_ai_v3'], conf_int[:, 0], conf_int[:, 1], color=color_map[species], alpha=0.2)
plt.yticks([-0.5, 0, 0.5])
plt.xlim(-1,1)
plt.xlabel('Log10(Aridity index)')
plt.ylabel('Growth loss',fontsize=18)
plt.legend(fontsize=10,loc=1)
plt.grid(True)
plt.savefig('FigS3b.jpg', dpi=300, bbox_inches='tight')
plt.show()
dfITRDB = pd.read_csv('gl_Biome.csv')
ITRDB=dfITRDB.loc[:,('Growth_loss','site')]
ITRDB.columns=['Growth_lossITRDB','site']
MODIS = df_merge95.loc[:,('Growth_loss','site')]
MODIS.columns=['Growth_lossMODSI','site']
cs1 = pd.merge(ITRDB,MODIS,on='site')
X = cs1['Growth_lossMODSI']
y = cs1['Growth_lossITRDB']
X = sm.add_constant(X)  
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
conf_int = model.conf_int(alpha=0.05)
predictions_summary_frame = model.get_prediction(X).summary_frame(alpha=0.05)
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)
p_values = model.pvalues
plt.figure(figsize=(5, 4.5))
plt.scatter(cs1['Growth_lossMODSI'], cs1['Growth_lossITRDB'], color=(0/255, 92/255, 230/255) ,alpha=0.2,)
plt.plot(cs1['Growth_lossMODSI'], predictions, color='red')
plt.fill_between(cs1['Growth_lossMODSI'], 
                 predictions_summary_frame['mean_ci_lower'], 
                 predictions_summary_frame['mean_ci_upper'], 
                 color='red', alpha=0.2)
plt.yticks([-0.2,0,0.2,0.4])
plt.xticks([-0.2,0,0.2,0.4])
plt.xlabel('Growth losses calculated by MODIS')
plt.ylabel('Growth loss calculated by ITRDB')
plt.text(0.05, 0.99, f'R² = {r2:.2f}\nMSE = {mse:.2f}\np-value = {p_values[1]:.2e}', 
         transform=plt.gca().transAxes, fontsize=16, verticalalignment='top')

plt.tight_layout()
plt.savefig('FigS3a.jpg', dpi=300, bbox_inches='tight')
plt.show()